<a href="https://colab.research.google.com/github/joshiayush/cardmatch/blob/main/notebooks/card_seek_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install langchain >/dev/null
!pip install langchain-community >/dev/null
!pip install langchain-openai >/dev/null
!pip install pypdf >/dev/null
!pip install chromadb >/dev/null

In [40]:
import os
import getpass

In [41]:
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LANGCHAIN_API_KEY:")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY:")

LANGCHAIN_API_KEY:··········
OPENAI_API_KEY:··········


In [42]:
import pathlib
from typing import Dict, Any

import numpy as np
import pandas as pd
from langchain.schema import Document, BaseRetriever
from langchain.document_loaders import (
    PyPDFLoader,
    TextLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredEPubLoader,
    WebBaseLoader,
)
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.base import Chain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [43]:
_cc_sheets = ["sbi", "axis", "hdfc"]


def load_cc_info() -> Dict:
    cc_info = dict()
    for sheet in _cc_sheets:
        df = pd.read_excel("cc_urls.xlsx", sheet_name=sheet)
        cc_info.update(df.set_index("card_name").T.to_dict())
    return cc_info

In [44]:
credit_cards_info = load_cc_info()

<ipython-input-43-481e6df65adc>:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  cc_info.update(df.set_index("card_name").T.to_dict())
<ipython-input-43-481e6df65adc>:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  cc_info.update(df.set_index("card_name").T.to_dict())


In [45]:
class DocumentLoaderException(Exception):
    pass

In [46]:
class EPubReader(UnstructuredEPubLoader):
    def __init__(self, file_path: str | list[str], **kwargs: Any):
        super().__init__(file_path, **kwargs, mode="elements", strategy="fast")

In [47]:
class DocumentLoader(object):
    """Loads in a document with a supported extension."""

    supported_extensions = {
        ".pdf": PyPDFLoader,
        ".txt": TextLoader,
        ".epub": EPubReader,
        ".docx": UnstructuredWordDocumentLoader,
        ".doc": UnstructuredWordDocumentLoader,
    }

In [48]:
def load_document(filepath: str) -> list[Document]:
  """Load a file and return it as a list of Document."""
  ext = pathlib.Path(filepath).suffix

  loader = DocumentLoader.supported_extensions.get(ext)
  if not loader:
    raise DocumentLoaderException(f"Unsupported file extension: {ext}")

  return loader(filepath).load()

In [49]:
def configure_retriever(docs: list[Document]) -> BaseRetriever:
    """Retriever to use."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    embeddings = OpenAIEmbeddings()
    vectordb = Chroma.from_documents(splits, embeddings)
    return vectordb.as_retriever(
        search_type="mmr", search_kwargs={"k": 2, "fetch_k": 4}
    )

In [50]:
def configure_chain(retriever: BaseRetriever) -> Chain:
    """Configure chain with a retriever."""
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
    return ConversationalRetrievalChain.from_llm(
        llm, retriever=retriever, memory=memory, verbose=True
    )

In [51]:
def configure_qa_chain(cc_info: Dict) -> Chain:
    """Read documents, configure retriever, and the chain."""
    docs = []
    for key, info in cc_info.items():
        if info["card_link"] is np.nan:
            continue

        card_doc = WebBaseLoader(info["card_link"]).load()
        docs.extend(card_doc)

        tnc_doc = load_document(info["tnc"])
        docs.extend(tnc_doc)
        break

    retriever = configure_retriever(docs)
    return configure_chain(retriever)

In [52]:
qa_chain = configure_qa_chain(credit_cards_info)

In [53]:
qa_chain.run("What is the name of the credit card?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Help me find a card


Track my Application


Retrieve my Application



Apply now


FAQ


Digital Platform Updates


Terms & Conditions



Connect with us


Facebook


twitter


instagram


linked-in


YouTube









CorporateCorporate Home


Customized Solution


Customized Solution Home



Corporate Web Servicing


Digital On-Boarding


Business Expertise


Global Acceptance


Technology Innovation


Security & Control


Best in Class Servicing


Solutions for You




Connect with us


Facebook


twitter


instagram


linked-in


YouTube







Cards


Cards Home



Corporate Cards


Central Travel Cards


Utility Cards


Purchase Cards


Virtual Cards



Enquire
Most Important Terms and Conditio

'The name of the credit card is SBI Card ELITE.'

In [54]:
qa_chain.run("What is the joining fee of this credit card?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the name of the credit card?
Assistant: The name of the credit card is SBI Card ELITE.
Follow Up Input: What is the joining fee of this credit card?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Annual Fee (one-time) : Rs.4,999 + Taxes
Welcome gift worth Rs. 5,000




Renewal Fee (per annum) : Rs. 4,999 + Taxes








Features of  SBI Card ELITE
Know everything about SBI Card ELITE



Features


Fees and Charges






Terms and Conditions

For detailed Terms and Conditions

'The joining fee of the SBI Card ELITE credit card is Rs. 4,999 plus taxes.'

In [55]:
qa_chain.run("What benefits do I get in movie tickets every month?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the name of the credit card?
Assistant: The name of the credit card is SBI Card ELITE.
Human: What is the joining fee of this credit card?
Assistant: The joining fee of the SBI Card ELITE credit card is Rs. 4,999 plus taxes.
Follow Up Input: What benefits do I get in movie tickets every month?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
View Card Statement


Utility Bill Payment


Blog






							Search
						









Apply Now


Get In Touch




EnglishEnglishहिन्दी



'With the SBI Card ELITE credit card, you get complimentary movie tickets worth Rs. 6,000 every year. Each month, you can avail a maximum discount of Rs. 250 per ticket for 2 tickets, with the transaction being valid for at least 2 tickets per booking. Note that a convenience fee would be chargeable.'